In [1]:
from autostep3.util import *
from autostep3 import better_aubio
from threading import Thread
import soundfile as sf
import sounddevice as sd
import numpy as np

Separate the drums from the song

In [2]:
drums_path = separate_drums("test_audio/hyperpop.mp3")

Generate onsets based on the drums
- Change the onset method to experiment

In [16]:
onsets = better_aubio.onset(drums_path, "mkl")
tempo = 130
print(tempo)

[mp3float @ 0x565497f74ac0] Could not update timestamps for skipped samples.


130


[mp3float @ 0x565497f74ac0] Could not update timestamps for discarded samples.


Load the separated drums for playback

In [11]:
drums, samplerate = sf.read(drums_path, dtype='float32')
num_samples = sf.info(drums_path).frames

**Experiment** (optional cell): snap onsets to nearest 16th notes

In [17]:
onsets = snap_sample(onsets, samplerate, tempo, 16)

Generate the "audible onsets" track; on every onset you will hear a very short clipping sound

In [18]:
# god numpy is FAST
audible_onsets = np.float32(np.isin(np.arange(num_samples), onsets))

Create 2 audio streams using `sounddevice` to play both tracks simultaneously

In [19]:
song_stream = sd.OutputStream(channels=2, samplerate=samplerate)
onsets_stream = sd.OutputStream(channels=1, samplerate=samplerate)

The streams need to be multithreaded, since `OutputStream.write()` is a blocking call.
This cell begins audio playback.

In [20]:
song_stream.start()
onsets_stream.start()
Thread(target=lambda: song_stream.write(drums)).start()
Thread(target=lambda: onsets_stream.write(audible_onsets)).start()

Run this cell to stop the streams, in case errors occur in the main thread

In [ ]:
song_stream.stop()
onsets_stream.stop()

Expression 'alsa_snd_pcm_mmap_begin( self->pcm, &areas, &self->offset, numFrames )' failed in 'src/hostapi/alsa/pa_linux_alsa.c', line: 3998
Expression 'PaAlsaStreamComponent_RegisterChannels( &self->playback, &self->bufferProcessor, &playbackFrames, &xrun )' failed in 'src/hostapi/alsa/pa_linux_alsa.c', line: 4118
Expression 'PaAlsaStream_SetUpBuffers( stream, &framesGot, &xrun )' failed in 'src/hostapi/alsa/pa_linux_alsa.c', line: 4495
Exception in thread Thread-9 (<lambda>):
Traceback (most recent call last):
  File "/home/t_/.pyenv/versions/3.10.13/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/t_/.pyenv/versions/3.10.13/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_5377/918457136.py", line 4, in <lambda>
  File "/home/t_/Projects/Autostep/.venv/lib/python3.10/site-packages/sounddevice.py", line 1544, in write
    return RawOutputStream.write(self, data)
  File "/home/t_